# 🌊 Kanyakumari Ocean Wave & Tsunami Prediction System

Run each cell in order!

In [ ]:
# Step 1: Clone repository and install dependencies
!git clone https://github.com/vsiva763-git/ocean-wave-disaster-prediction.git 2>/dev/null || (cd ocean-wave-disaster-prediction && git pull)
%cd ocean-wave-disaster-prediction
!pip install fastapi uvicorn httpx feedparser pandas numpy jinja2 python-multipart pyngrok -q
print("✅ Setup complete!")

In [ ]:
# Step 2: Test data fetching
import sys
sys.path.insert(0, 'src')
from kanyakumari_monitor import KanyakumariOceanMonitor
monitor = KanyakumariOceanMonitor()
print("🔄 Fetching real-time data...")
marine = monitor.fetch_marine_data()
weather = monitor.fetch_weather_data()
earthquakes = monitor.fetch_earthquakes()
current = marine.get('current', {})
print(f"🌊 Wave Height: {current.get('wave_height_m', 'N/A')} m")
print(f"🌡️ Temperature: {weather.get('temperature_c', 'N/A')}°C")
print(f"�� Earthquakes: {earthquakes.get('total_count', 0)}")
print("✅ APIs working!")

In [ ]:
# Step 3: Start FastAPI server
import subprocess, threading, time, requests
def run_server():
    subprocess.run(['python', '-m', 'uvicorn', 'src.kanyakumari_api:app', '--host', '0.0.0.0', '--port', '8000'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(5)
try:
    r = requests.get('http://127.0.0.1:8000/api/status', timeout=10)
    print("✅ FastAPI server running on port 8000")
except Exception as e:
    print(f"❌ Server error: {e}")

In [ ]:
# Step 4: Create public URL with ngrok
from pyngrok import ngrok
import time
ngrok.set_auth_token("2qFZUpn9LPoFoI2QDT3HwNdE_6t2Kg24ewa4QuEdSDa9fx")
ngrok.kill()
time.sleep(1)
public_url = ngrok.connect(8000)
print("=" * 60)
print("🌐 PUBLIC URL (Click to open dashboard):")
print(f"   {public_url.public_url}")
print("=" * 60)

In [ ]:
# Step 5: Test prediction API
import requests
response = requests.get('http://127.0.0.1:8000/api/predict')
data = response.json()
if data.get('success'):
    pred = data.get('prediction', {})
    marine = data.get('current_conditions', {}).get('marine', {})
    risk = data.get('overall_risk', {})
    print(f"🌊 Wave Height: {marine.get('wave_height_m', 'N/A')} m")
    print(f"🌊 Severity: {pred.get('wave_severity', 'N/A')}")
    print(f"⚠️ Risk Level: {risk.get('overall_risk_level', 'N/A')}")